In [ ]:

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras
import tensorflow_hub as hub
import tensorflow_datasets as tfds
from functools import partial
from tensorflow.keras.applications import EfficientNetB0

In [ ]:
'''try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver.connect()
    print("Device:", tpu.master())
    strategy = tf.distribute.TPUStrategy(tpu)
except ValueError:
    print("Not connected to a TPU runtime. Using CPU/GPU strategy")
    strategy = tf.distribute.MirroredStrategy()'''

'try:\n    tpu = tf.distribute.cluster_resolver.TPUClusterResolver.connect()\n    print("Device:", tpu.master())\n    strategy = tf.distribute.TPUStrategy(tpu)\nexcept ValueError:\n    print("Not connected to a TPU runtime. Using CPU/GPU strategy")\n    strategy = tf.distribute.MirroredStrategy()'

In [ ]:

fashion_mnist = tf.keras.datasets.fashion_mnist.load_data()
(X_train_full, y_train_full), (X_test, y_test) = fashion_mnist

X_train_full = np.expand_dims(X_train_full, axis=-1).astype(np.float32) / 255
X_test = np.expand_dims(X_test.astype(np.float32), axis=-1) / 255

X_train, X_valid = X_train_full[:-5000], X_train_full[-5000:]
y_train, y_valid = y_train_full[:-5000], y_train_full[-5000:]

In [ ]:
tf.random.set_seed(42)

In [ ]:
X_train.shape, X_valid.shape, X_test.shape, y_train.shape

((55000, 28, 28, 1), (5000, 28, 28, 1), (10000, 28, 28, 1), (55000,))

In [ ]:
DefaultConv2D = partial(tf.keras.layers.Conv2D, kernel_size=3, padding="same", activation="relu", kernel_initializer="he_normal")

model = tf.keras.Sequential([
    DefaultConv2D(filters=64, kernel_size=7, input_shape=[28, 28, 1]),
    tf.keras.layers.MaxPool2D(),

    DefaultConv2D(filters=128),
    DefaultConv2D(filters=128),
    tf.keras.layers.MaxPool2D(),

    DefaultConv2D(filters=256),
    DefaultConv2D(filters=256),
    tf.keras.layers.MaxPool2D(),

    tf.keras.layers.Flatten(),
    
    tf.keras.layers.Dense(units=128, activation="relu", kernel_initializer="he_normal"),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(units=64, activation="relu", kernel_initializer="he_normal"),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(units=10, activation="softmax")
    ])

In [ ]:
model.compile(loss="sparse_categorical_crossentropy", optimizer="nadam", metrics=["accuracy"])

history = model.fit(X_train, y_train, epochs=3, validation_data=(X_valid, y_valid))

score = model.evaluate(X_test, y_test)

Epoch 1/3
1719/1719 [==============================] - 31s 9ms/step - loss: 0.7104 - accuracy: 0.7520 - val_loss: 0.3588 - val_accuracy: 0.8650
Epoch 2/3
1719/1719 [==============================] - 16s 9ms/step - loss: 0.4041 - accuracy: 0.8659 - val_loss: 0.2969 - val_accuracy: 0.8850
Epoch 3/3
313/313 [==============================] - 1s 5ms/step - loss: 0.3054 - accuracy: 0.8956


In [ ]:

X_test_samples = X_test[:10] 
y_pred = model.predict(X_test_samples)

1/1 [==============================] - 0s 326ms/step


## LeNet-5


In [ ]:
X_train_ = tf.image.resize(X_train, [32, 32])
X_train_ = tf.cast(X_train_, dtype=tf.float32) / 255.0

X_valid_ = tf.image.resize(X_valid, [32, 32])
X_valid_ = tf.cast(X_valid_, dtype=tf.float32) / 255.0

X_test_ = tf.image.resize(X_test, [32, 32])
X_test_ = tf.cast(X_test_, dtype=tf.float32) / 255.0

In [ ]:
model = keras.models.Sequential()
model.add(keras.layers.Conv2D(filters=32, kernel_size=(5,5), padding='same', activation='relu', input_shape=(32, 32, 1)))
model.add(keras.layers.MaxPool2D(strides=2))
model.add(keras.layers.Conv2D(filters=48, kernel_size=(5,5), padding='valid', activation='relu'))
model.add(keras.layers.MaxPool2D(strides=2))
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(256, activation='relu'))
model.add(keras.layers.Dense(84, activation='relu'))
model.add(keras.layers.Dense(10, activation='softmax'))

In [ ]:
model.compile(loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

history = model.fit(X_train_, y_train, epochs=10, validation_data=(X_valid_, y_valid))

Epoch 1/10
1719/1719 [==============================] - 11s 5ms/step - loss: 2.3028 - accuracy: 0.0975 - val_loss: 2.3026 - val_accuracy: 0.0980
Epoch 2/10
1719/1719 [==============================] - 8s 5ms/step - loss: 2.3028 - accuracy: 0.0968 - val_loss: 2.3026 - val_accuracy: 0.1054
Epoch 3/10
1719/1719 [==============================] - 9s 5ms/step - loss: 2.3028 - accuracy: 0.0987 - val_loss: 2.3026 - val_accuracy: 0.0934
Epoch 4/10
1719/1719 [==============================] - 8s 4ms/step - loss: 2.3028 - accuracy: 0.0991 - val_loss: 2.3025 - val_accuracy: 0.1016
Epoch 5/10
1719/1719 [==============================] - 8s 5ms/step - loss: 2.3028 - accuracy: 0.0988 - val_loss: 2.3029 - val_accuracy: 0.1006
Epoch 6/10
1719/1719 [==============================] - 10s 6ms/step - loss: 2.3028 - accuracy: 0.0982 - val_loss: 2.3030 - val_accuracy: 0.0900
Epoch 7/10
1719/1719 [==============================] - 8s 4ms/step - loss: 2.3027 - accuracy: 0.1025 - val_loss: 2.3024 - val_accurac

In [ ]:
score = model.evaluate(X_test_, y_test)
score

313/313 [==============================] - 1s 3ms/step - loss: 2.3027 - accuracy: 0.1000


[2.302663564682007, 0.10000000149011612]

## Transfer Learning with Efficient Net

In [ ]:
model = EfficientNetB0(include_top=False, weights='imagenet')

16705208/16705208 [==============================] - 1s 0us/step


In [ ]:
IMG_SIZE = 224
BATCH_SIZE = 32
NUM_CLASSES = 10

In [ ]:
train_ds = tf.data.Dataset.from_tensor_slices((X_train, y_train))
test_ds = tf.data.Dataset.from_tensor_slices((X_test, y_test))

In [ ]:
size = (IMG_SIZE, IMG_SIZE)
train_ds = train_ds.map(lambda image, label: (tf.image.resize(image, size), label))
test_ds = test_ds.map(lambda image, label: (tf.image.resize(image, size), label))

In [ ]:
def input_preprocess(image, label):
    label = tf.one_hot(label, NUM_CLASSES)
    return image, label

train_ds = train_ds.map(input_preprocess, num_parallel_calls=tf.data.AUTOTUNE)
train_ds = train_ds.batch(batch_size=BATCH_SIZE, drop_remainder=True)
train_ds = train_ds.prefetch(tf.data.AUTOTUNE)

test_ds = test_ds.map(input_preprocess)
test_ds = test_ds.batch(batch_size=BATCH_SIZE, drop_remainder=True)

In [ ]:
def build_model(num_classes):
    inputs = keras.layers.Input(shape=(IMG_SIZE, IMG_SIZE, 3))

    model = EfficientNetB0(include_top=False, input_tensor=inputs, weights="imagenet")
    model.trainable = False

    x = keras.layers.GlobalAveragePooling2D(name="avg_pool")(model.output)
    x = keras.layers.BatchNormalization()(x)
    top_dropout_rate = 0.2
    x = keras.layers.Dropout(top_dropout_rate, name="top_dropout")(x)
    outputs = keras.layers.Dense(NUM_CLASSES, activation="softmax", name="pred")(x)

    model = tf.keras.Model(inputs, outputs, name="EfficientNet")
    optimizer = tf.keras.optimizers.Adam(learning_rate=1e-2)
    model.compile(optimizer=optimizer, loss="categorical_crossentropy", metrics=["accuracy"])
    return model

In [ ]:
model = build_model(num_classes=NUM_CLASSES)

hist = model.fit(train_ds, epochs=3, validation_data=test_ds, verbose=2)

Epoch 1/3
1718/1718 - 115s - loss: 2.4636 - accuracy: 0.1003 - val_loss: 2.3050 - val_accuracy: 0.1001 - 115s/epoch - 67ms/step
Epoch 2/3
1718/1718 - 107s - loss: 2.3202 - accuracy: 0.1012 - val_loss: 2.3068 - val_accuracy: 0.1000 - 107s/epoch - 62ms/step
Epoch 3/3
1718/1718 - 108s - loss: 2.3395 - accuracy: 0.1018 - val_loss: 2.3065 - val_accuracy: 0.1000 - 108s/epoch - 63ms/step


## Transfer Learning with BiT

In [ ]:

IMG_SIZE = 384
CROP_TO = 224
BATCH_SIZE = 64
STEPS_PER_EPOCH = 10
AUTO = tf.data.AUTOTUNE  
NUM_CLASSES = 10
SCHEDULE_LENGTH = (20)
SCHEDULE_BOUNDARIES = [5, 10, 15]

In [ ]:
train_ds = tf.data.Dataset.from_tensor_slices((X_train, y_train))
test_ds = tf.data.Dataset.from_tensor_slices((X_test, y_test))

In [ ]:

@tf.function
def preprocess_train(image, label):
    image = tf.image.random_flip_left_right(image)
    image = tf.image.resize(image, (CROP_TO, CROP_TO))
    image = tf.image.grayscale_to_rgb(image)
    image = tf.image.random_crop(image, (CROP_TO, CROP_TO, 3))
    image = image / 255.0
    return (image, label)

@tf.function
def preprocess_test(image, label):
    image = tf.image.resize(image, (CROP_TO, CROP_TO))
    image = tf.image.grayscale_to_rgb(image)
    image = image / 255.0
    return (image, label)

DATASET_NUM_TRAIN_EXAMPLES = train_ds.cardinality().numpy()

repeat_count = int(SCHEDULE_LENGTH * BATCH_SIZE / DATASET_NUM_TRAIN_EXAMPLES * STEPS_PER_EPOCH)
repeat_count += 10 + 1

In [ ]:
train_ds = train_ds.shuffle(10000)
train_ds = train_ds.repeat(repeat_count)
train_ds = train_ds.map(preprocess_train, num_parallel_calls=AUTO)
train_ds = train_ds.batch(BATCH_SIZE)
train_ds = train_ds.prefetch(AUTO)

test_ds = test_ds.map(preprocess_test, num_parallel_calls=AUTO)
test_ds = test_ds.batch(BATCH_SIZE)
test_ds = test_ds.prefetch(AUTO)

In [ ]:

bit_model_url = "https://tfhub.dev/google/bit/m-r50x1/1"
bit_module = hub.KerasLayer(bit_model_url)

In [ ]:
class BiTModel(keras.Model):
  def __init__(self, num_classes, module, **kwargs):
    super().__init__(**kwargs)
    self.num_classes = num_classes
    self.head = keras.layers.Dense(num_classes, kernel_initializer="zeros")
    self.bit_model = module

  def call(self, images):
    bit_embedding = self.bit_model(images)
    return self.head(bit_embedding)

model = BiTModel(num_classes=NUM_CLASSES, module=bit_module)

In [ ]:

learning_rate = 0.005 * BATCH_SIZE / 512

lr_schedule = keras.optimizers.schedules.PiecewiseConstantDecay(
    boundaries=SCHEDULE_BOUNDARIES,
    values=[
        learning_rate,
        learning_rate * 0.1,
        learning_rate * 0.01,
        learning_rate * 0.001,
    ],
)
optimizer = keras.optimizers.SGD(learning_rate=lr_schedule, momentum=0.9)

loss_fn = keras.losses.SparseCategoricalCrossentropy(from_logits=True)

In [ ]:
model.compile(optimizer=optimizer, loss=loss_fn, metrics=["accuracy"])

In [ ]:

callbacks = [keras.callbacks.EarlyStopping(monitor="val_accuracy", patience=2, restore_best_weights=True)]

In [ ]:
history = model.fit(
    train_ds,
    batch_size=BATCH_SIZE,
    epochs=3,
    steps_per_epoch=STEPS_PER_EPOCH,
    validation_data=test_ds,
    callbacks=callbacks,
)

Epoch 1/3
10/10 [==============================] - 97s 10s/step - loss: nan - accuracy: 0.1047 - val_loss: nan - val_accuracy: 0.1000
Epoch 2/3
10/10 [==============================] - 86s 9s/step - loss: nan - accuracy: 0.1109 - val_loss: nan - val_accuracy: 0.1000
Epoch 3/3
10/10 [==============================] - 86s 9s/step - loss: nan - accuracy: 0.0797 - val_loss: nan - val_accuracy: 0.1000


In [ ]:

accuracy = model.evaluate(test_ds)[1] * 100
print("Accuracy: {:.2f}%".format(accuracy))

157/157 [==============================] - 58s 368ms/step - loss: nan - accuracy: 0.1000
Accuracy: 10.00%


## Image augmentation

In [ ]:
train_ds = tf.data.Dataset.from_tensor_slices((X_train, y_train))
test_ds = tf.data.Dataset.from_tensor_slices((X_test, y_test))

In [ ]:
train_images = train_ds.map(lambda x, y: x).as_numpy_iterator()
train_labels = train_ds.map(lambda x, y: y).as_numpy_iterator()

plt.figure(figsize=(6, 6))
for i, (image, label) in enumerate(zip(train_images, train_labels)):
    if i >= 9:
        break
    plt.subplot(3, 3, i + 1)
    plt.imshow(image, cmap='gray')
    plt.title(label)
    plt.axis('off')

plt.tight_layout()
plt.show()

In [ ]:
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomFlip(mode="horizontal", seed=42),
    tf.keras.layers.RandomRotation(factor=0.05, seed=42),
    tf.keras.layers.RandomContrast(factor=0.2, seed=42)
])

In [ ]:
augmented_train_ds = train_ds.map(lambda x, y: (data_augmentation(x, training=True), y))

augmented_images = []
for image, label in augmented_train_ds.take(9):
    augmented_images.append(tf.squeeze(image))

plt.figure(figsize=(6, 6))
for i, image in enumerate(augmented_images):
    plt.subplot(3, 3, i + 1)
    plt.imshow(image, cmap='gray')
    plt.axis('off')

plt.tight_layout()
plt.show()

## MLP Mixer

In [ ]:
num_classes = 10
input_shape = (32, 32, 3)

weight_decay = 0.0001
batch_size = 64
num_epochs = 3
dropout_rate = 0.2
image_size = 32  
patch_size = 8  
num_patches = (image_size // patch_size) ** 2 
embedding_dim = 256  
num_blocks = 4  
learning_rate = 0.005

In [ ]:
train_ds = tf.data.Dataset.from_tensor_slices((X_train, y_train))
test_ds = tf.data.Dataset.from_tensor_slices((X_test, y_test))

In [ ]:
def transforms(image, label):
    image = tf.image.resize(image, [32, 32])
    image = tf.image.grayscale_to_rgb(image)
    image = image / 255.0
    return image, label

train_ds = train_ds.map(transforms)

test_ds = test_ds.map(transforms)

In [ ]:
class Patches(keras.layers.Layer):
  def __init__(self, patch_size, num_patches):
    super().__init__()
    self.patch_size = patch_size
    self.num_patches = num_patches

  def call(self, images):
    batch_size = tf.shape(images)[0]
    patches = tf.image.extract_patches(
        images=images,
        sizes=[1, self.patch_size, self.patch_size, 1],
        strides=[1, self.patch_size, self.patch_size, 1],
        rates=[1, 1, 1, 1],
        padding="VALID",
    )
    patch_dims = patches.shape[-1]
    patches = tf.reshape(patches, [batch_size, self.num_patches, patch_dims])
    return patches

In [ ]:

def build_classifier(blocks, positional_encoding=False):
    inputs = keras.layers.Input(shape=input_shape)
    
    patches = Patches(patch_size, num_patches)(inputs)

    x = keras.layers.Dense(units=embedding_dim)(patches)

    if positional_encoding:
        positions = tf.range(start=0, limit=num_patches, delta=1)
        position_embedding = keras.layers.Embedding(
            input_dim=num_patches, output_dim=embedding_dim
        )(positions)
        x = x + position_embedding

    x = blocks(x)
    representation = keras.layers.GlobalAveragePooling1D()(x)
    representation = keras.layers.Dropout(rate=dropout_rate)(representation)
    logits = keras.layers.Dense(num_classes)(representation)

    return keras.Model(inputs=inputs, outputs=logits)

In [ ]:
def run_experiment(model):
    optimizer = keras.optimizers.AdamW(learning_rate=learning_rate, weight_decay=weight_decay)
    model.compile(
        optimizer=optimizer,
        loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        metrics=[
            keras.metrics.SparseCategoricalAccuracy(name="acc"),
            keras.metrics.SparseTopKCategoricalAccuracy(5, name="top5-acc"),
        ],
    )

    reduce_lr = keras.callbacks.ReduceLROnPlateau(
        monitor="val_loss", factor=0.5, patience=5
    )
    early_stopping = tf.keras.callbacks.EarlyStopping(
        monitor="val_loss", patience=10, restore_best_weights=True
    )


    train_ds_expanded = train_ds.map(lambda x, y: (tf.expand_dims(x, axis=0), y))
    test_ds_expanded = test_ds.map(lambda x, y: (tf.expand_dims(x, axis=0), y))

    history = model.fit(
        train_ds_expanded,
        batch_size=batch_size,
        epochs=num_epochs,
        callbacks=[early_stopping, reduce_lr],
    )

    _, accuracy, top_5_accuracy = model.evaluate(test_ds_expanded)
    print(f"Test accuracy: {round(accuracy * 100, 2)}%")
    print(f"Test top 5 accuracy: {round(top_5_accuracy * 100, 2)}%")

    return history

In [ ]:
class MLPMixerLayer(keras.layers.Layer):
  def __init__(self, num_patches, hidden_units, dropout_rate, *args, **kwargs):
    super().__init__(*args, **kwargs)

    self.mlp1 = keras.Sequential(
        [
            keras.layers.Dense(units=num_patches),
            #keras.tfa.layers.GELU(),
            keras.layers.Dense(units=num_patches),
            keras.layers.Dropout(rate=dropout_rate),
        ]
    )
    self.mlp2 = keras.Sequential(
        [
            keras.layers.Dense(units=num_patches),
            #keras.tfa.layers.GELU(),
            keras.layers.Dense(units=embedding_dim),
            keras.layers.Dropout(rate=dropout_rate),
        ]
    )
    self.normalize = keras.layers.LayerNormalization(epsilon=1e-6)

  def call(self, inputs):
    x = self.normalize(inputs)
    x_channels = tf.linalg.matrix_transpose(x)
    mlp1_outputs = self.mlp1(x_channels)
    mlp1_outputs = tf.linalg.matrix_transpose(mlp1_outputs)
    x = mlp1_outputs + inputs
    x_patches = self.normalize(x)
    mlp2_outputs = self.mlp2(x_patches)
    x = x + mlp2_outputs
    return x

In [ ]:
mlpmixer_blocks = keras.Sequential([MLPMixerLayer(num_patches, embedding_dim, dropout_rate) for _ in range(num_blocks)])

mlpmixer_classifier = build_classifier(mlpmixer_blocks)
history = run_experiment(mlpmixer_classifier)

Epoch 1/3


ValueError: ignored